# Build regression model: Start wit R and Tidymodels for regression model


## Introduction to Regression - Lesson 1

#### Make we put am for better understanding

✅ E get plenty kain regression methods, and di one wey you go choose depend on di kind ansa wey you dey find. If you wan predict di possible height for person wey get one kind age, you go use `linear regression`, because na **number value** you dey look for. But if na to know whether one kind food na vegan or e no be vegan, na **category assignment** you dey find, so you go use `logistic regression`. You go sabi more about logistic regression later. Try think about some kind questions wey you fit ask data, and which of these methods go fit pass.

For dis section, you go work with [small dataset about diabetes](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html). Imagine say you wan test one treatment for people wey get diabetes. Machine Learning models fit help you know which patients go respond well to di treatment, based on di combination of variables. Even di most simple regression model, if you see am for graph, fit show you information about variables wey go help you plan your clinical trials well.

So, make we start dis task!

<p >
   <img src="../../../../../../translated_images/pcm/encouRage.e75d5fe0367fb913.webp"
   width="630"/>
   <figcaption>Artwork by @allison_horst</figcaption>


## 1. Load our tool set

For dis task, we go need dis packages:

-   `tidyverse`: Di [tidyverse](https://www.tidyverse.org/) na [collection of R packages](https://www.tidyverse.org/packages) wey dem design to make data science fast, easy, and fun!

-   `tidymodels`: Di [tidymodels](https://www.tidymodels.org/) framework na [collection of packages](https://www.tidymodels.org/packages/) for modeling and machine learning.

You fit install dem like dis:

`install.packages(c("tidyverse", "tidymodels"))`

Di script wey dey below go check whether you get di packages wey you need to complete dis module, and e go install dem for you if any dey miss.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



Make we load dis beta packages dem and make dem dey available for our current R session. (Dis na just for example, `pacman::p_load()` don already do am for you)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. Diabetes dataset

For dis exercise, we go show our regression skills by predicting tins for diabetes dataset. Di [diabetes dataset](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) get `442 samples` of data wey dey about diabetes, wit 10 predictor feature variables, `age`, `sex`, `body mass index`, `average blood pressure`, and `six blood serum measurements` plus one outcome variable `y`: one quantitative measure of how di disease don progress one year after baseline.

|Number of observations|442|
|----------------------|:---|
|Number of predictors|First 10 columns na numeric predictive|
|Outcome/Target|Column 11 na quantitative measure of how di disease don progress one year after baseline|
|Predictor Information|- age in years
||- sex
||- bmi body mass index
||- bp average blood pressure
||- s1 tc, total serum cholesterol
||- s2 ldl, low-density lipoproteins
||- s3 hdl, high-density lipoproteins
||- s4 tch, total cholesterol / HDL
||- s5 ltg, maybe log of serum triglycerides level
||- s6 glu, blood sugar level|




> 🎓 Make sure say you remember, dis na supervised learning, and we need one target wey dem call 'y'.

Before you fit do any manipulation for data wit R, you go first import di data enter R memory, or you go build connection wey R fit use access di data remotely.

> Di [readr](https://readr.tidyverse.org/) package, wey dey part of Tidyverse, dey provide fast and easy way to read rectangular data enter R.

Now, make we load di diabetes dataset wey dey dis source URL: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

We go also check our data small to make sure say e dey okay using `glimpse()` and show di first 5 rows using `slice()`.

Before we go further, make we introduce one tin wey you go dey see well well for R code 🥁🥁: di pipe operator `%>%`

Di pipe operator (`%>%`) dey help perform operations in logical sequence by passing one object go front enter one function or call expression. You fit think of di pipe operator like say e dey talk "and then" for your code.


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` dey show say dis data get 442 rows and 11 columns, and all di columns na `double` type. 

<br>

> glimpse() and slice() na functions wey dey inside [`dplyr`](https://dplyr.tidyverse.org/). Dplyr, wey be part of Tidyverse, na grammar for data manipulation wey dey give consistent set of verbs wey go help you solve di common wahala for data manipulation.

<br>

Now wey we don get di data, make we focus on one feature (`bmi`) wey we go use for dis exercise. To do dis one, we go need select di column wey we want. So, how we go fit do am?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) dey allow us *select* (and if we want, rename) columns for inside data frame.


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. Training and Testing data

For supervised learning, e dey normal to *divide* di data into two parts; one big part wey dem go use train di model, and one small "hold-back" part wey dem go use check how di model take perform.

Now wey we don ready di data, we fit see if machine fit help us decide beta way to divide di numbers for dis dataset. We fit use di [rsample](https://tidymodels.github.io/rsample/) package, wey be part of di Tidymodels framework, to create one object wey go hold di information on *how* to divide di data, and then use two more rsample functions to bring out di training and testing sets wey we don create:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Train linear regression model wit Tidymodels

Now we don ready to train our model!

For Tidymodels, you go use `parsnip()` to set up model by specify three things:

-   Model **type** na wetin go make models different, like linear regression, logistic regression, decision tree models, and so on.

-   Model **mode** na common options like regression and classification; some model types fit for do any of these, while some na only one mode dem get.

-   Model **engine** na the tool wey go do the calculation to fit the model. Most times, na R packages dem be, like **`"lm"`** or **`"ranger"`**

Dis modeling info dey inside model specification, so make we build one!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

Afta dem don *specify* di model, di model fit dey `estimate` or `train` wit di [`fit()`](https://parsnip.tidymodels.org/reference/fit.html) function, wey dem dey usually use formula and some data.

`y ~ .` mean say we go fit `y` as di tin wey we wan predict/di target, wey all di predictors/features go explain am, i.e., `.` (for dis case, we get only one predictor: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

From wetin de model show us, we fit see di coefficients wey e learn during training. Dem represent di coefficients of di line wey fit pass well well wey go give us di lowest overall error between di real and di predicted variable.  
<br>

## 5. Predict for di test set

Now we don train di model, we fit use am predict how di disease go waka (y) for di test dataset using [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Dis one go help us draw di line wey go separate di data groups.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

Woohoo! 💃🕺 We don train one model and use am take make predictions!

Wen we dey make predictions, di tidymodels way na to always produce one tibble/data frame of results wey get standard column names. Dis one go make am easy to join di original data and di predictions for one format wey fit work well for di next steps like plotting.

`dplyr::bind_cols()` dey join multiple data frames column by column sharp sharp.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. Plot modelling results

Now na time to see dis one for eye 📈. We go create scatter plot of all di `y` and `bmi` values wey dey test set, den use di predictions take draw one line for di best place, between di model data groupings.

R get plenty systems to take make graphs, but `ggplot2` na one of di most fine and flexible ones. E go allow you fit **combine independent components** to take build your graphs.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ Make you think small about wetin dey happen here. One straight line dey pass through plenty small dots of data, but wetin e dey really do? You fit see how you go fit use dis line take predict where new data wey you never see before go fit enter for the plot y axis? Try talk the practical use of dis model for words.

Congrats, you don build your first linear regression model, use am predict something, and show am for one plot!


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Dis dokyument don use AI transleto service [Co-op Translator](https://github.com/Azure/co-op-translator) do di translation. Even as we dey try make am accurate, abeg make you sabi say machine translation fit get mistake or no dey correct well. Di original dokyument for im native language na di main source wey you go fit trust. For important mata, e good make professional human translator check am. We no go fit take blame for any misunderstanding or wrong interpretation wey fit happen because you use dis translation.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
